# Technical Trading with Interactive Brokers

- Create the Trading Implementation for a 50/200 Days SMA Crossover Strategy for MSFT with Interactive Brokers.
- Develop the code and create a Python Script that can be executed at the end of each trading day.

---

# Get Data and create Technical Indicators

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime

In [2]:
symbol = "MSFT"

In [3]:
shares = 10

In [4]:
sma_s = 50
sma_l = 200

In [5]:
today = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))
today

Timestamp('2023-11-14 00:00:00')

In [ ]:
cprice =  yf.Ticker(symbol).get_info()["regularMarketPrice"]
cprice

In [7]:
close = yf.download(symbol, end = today).Close.to_frame()
close

[*********************100%%**********************]  1 of 1 completed


,Close
Date,
1986-03-13,0.097222
1986-03-14,0.100694
1986-03-17,0.102431
1986-03-18,0.099826
1986-03-19,0.098090
...,...
2023-11-07,360.529999
2023-11-08,363.200012
2023-11-09,360.690002


In [ ]:
close.loc[today] = cprice
close

In [9]:
close = close.iloc[-sma_l:].copy()
close

,Close
Date,
2023-01-30,242.710007
2023-01-31,247.809998
2023-02-01,252.750000
2023-02-02,264.600006
2023-02-03,258.350006
...,...
2023-11-07,360.529999
2023-11-08,363.200012
2023-11-09,360.690002


In [ ]:
close["sma_s"] = close.Close.rolling(sma_s).mean()
close["sma_l"] = close.Close.rolling(sma_l).mean()

In [ ]:
close

---

# Identify target Position

In [24]:
close["position"] = np.where(close["sma_s"] > close["sma_l"], 1, -1 )
close

KeyError: 'sma_s'

In [ ]:
position = close.position.iloc[-1]
position

In [ ]:
target = position * shares
target

---

# Identify current Positions

In [10]:
from ib_insync import *
util.startLoop()

In [11]:
ib = IB()

In [12]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [13]:
pos = ib.positions()
pos

[Position(account='DU8048713', contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), position=1.0, avgCost=436.68),
 Position(account='DU8048713', contract=Stock(conId=442464659, symbol='OTRKP', exchange='PINK', currency='USD', localSymbol='OTRKP', tradingClass='CURRENT'), position=1.0, avgCost=0.3021)]

In [14]:
df = util.df(pos)
df

,account,contract,position,avgCost
0,DU8048713,"Stock(conId=756733, symbol='SPY', exchange='AR...",1.0,436.6800
1,DU8048713,"Stock(conId=442464659, symbol='OTRKP', exchang...",1.0,0.3021


In [15]:
if df is not None:
    df["symbol"] = df.contract.apply(lambda x: x.symbol)
    df["conID"] = df.contract.apply(lambda x: x.conId)
else: 
    df = pd.DataFrame(columns = ["symbol", "position", "conID"])

In [16]:
df

,account,contract,position,avgCost,symbol,conID
0,DU8048713,"Stock(conId=756733, symbol='SPY', exchange='AR...",1.0,436.6800,SPY,756733
1,DU8048713,"Stock(conId=442464659, symbol='OTRKP', exchang...",1.0,0.3021,OTRKP,442464659


---

# Identify Trades

In [17]:
# Get the contract (microsoft)
contract = Stock(symbol, "SMART", "USD")
contract

Stock(symbol='MSFT', exchange='SMART', currency='USD')

In [18]:
# Request more details
cds = ib.reqContractDetails(contract)
cds

[ContractDetails(contract=Contract(secType='STK', conId=272093, symbol='MSFT', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='MSFT', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,BENCHPX,CASHQTY,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,DUR,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,PEGMID,POSTATS,POSTONLY,PREOPGRTH,PRICECHK,REL,REL2MID,RELPCTOFS,RPI,RTH,SCALE,SCALEODD,SCALERST,SIZECHK,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,JEFFALGO,BYX,IEX,EDGX,FOXRIVER,PEARL,NYSENAT,LTSE,MEMX,IBEOS,OVERNIGHT,PSX', priceMagnifier=1, underConId=0, longName='MICROSOFT CORP', contractMonth='', industry='Technology', category='Software', subcategory='Applications Softw

In [19]:
conID = cds[0].contract.conId
conID

272093

In [20]:
# Check if there are positions
current_pos = df[df.conID == conID]
current_pos

,account,contract,position,avgCost,symbol,conID


In [21]:
# handle positions
if len(current_pos) == 0:
    actual = 0
else: 
    actual = current_pos.position.iloc[0]

In [22]:
actual

0

In [23]:
# update the trade, based on the actual position
trades = target - actual
trades

NameError: name 'target' is not defined

---

# Execute Trades

In [ ]:
if trades > 0:
    side = "BUY"
    order = MarketOrder(side, abs(trades))
    trade = ib.placeOrder(contract, order)
    while not trade.isDone():
        ib.waitOnUpdate()
    if trade.orderStatus.status == "Filled":
        print("{} {} {} @ {}".format(side, trade.orderStatus.filled, symbol, trade.orderStatus.avgFillPrice))
    else:
        print("{} {} failed.".format(side, symbol))
elif trades < 0:
    side = "SELL"
    order = MarketOrder(side, abs(trades))
    trade = ib.placeOrder(contract, order)
    while not trade.isDone():
        ib.waitOnUpdate()
    if trade.orderStatus.status == "Filled":
        print("{} {} {} @ {}".format(side, trade.orderStatus.filled, symbol, trade.orderStatus.avgFillPrice))
    else:
        
        print("{} {} failed.".format(side, symbol))
else:
    print("No Trades required.")

In [ ]:
ib.disconnect()